In [1]:
import requests
from bs4 import BeautifulSoup
import psycopg2

In [2]:
PG = {
    "host" : "localhost",
    "port" : "5432",
    "dbname" : "postgres",
    "user" : "postgres",
    "password" : "ja52712"
}
BASE_URL = "https://books.toscrape.com/"

In [14]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="ja52712",
    port="5432"
)
print("Postgresに接続できた")



Postgresに接続できた


In [15]:
cursor = conn.cursor()

In [19]:
create_table_sql = """
create table if not exists users(
    id serial primary key,
    title text not null,
    product_url text unique not null
);
"""


conn.commit()

In [20]:
cursor.execute(create_table_sql)
conn.commit()
print("usersテーブル作成完了")

usersテーブル作成完了


In [21]:
cursor.execute("select * from users;")
rows = cursor.fetchall()
print(rows)

[]


In [22]:
cursor.execute(
    "insert into users (title,product_url) values (%s,%s)",
    ("Test Book","https://books.toscrape.com/catalogue/test-book_1/index.html")
)
conn.commit()

print("データが追加できた")

データが追加できた


In [23]:
cursor.execute("select * from users;")
rows = cursor.fetchall()
print(rows)

[(1, 'Test Book', 'https://books.toscrape.com/catalogue/test-book_1/index.html')]
